In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid", {'legend.frameon':True})

columns = [
    'id', 'testing_for', 'address', 'zip_code', 'provider',
    'connected_with', 'monthly_price', 'provider_down_speed',
    'provider_price', 'actual_down_speed', 'actual_price', 'rating',
    'completed', 'created_at', 'updated_at', 'latitude', 'longitude',
    'ping', 'actual_upload_speed', 'test_id', 'ip_address', 'hostname',
    'from_mlab', 'area_code', 'test_type', 'census_code',
    'upload_median', 'download_median', 'census_status', 'test_date',
    'country_code', 'region', 'county', 'accuracy', 'location',
    'census_block'
]

# Load and filter CSV in chunks (less memory, require 5GB to load full dataset) to just from_mlab = 0 (SUA submissions)
iter_csv = pd.read_csv('./data/5d164a6e-6099-4824-9540-5bbc91327517.csv', header=None, names=columns, iterator=True, chunksize=1000)
sua = pd.concat([chunk[chunk['from_mlab'] == 0] for chunk in iter_csv])

# Cleanup
sua['rating'] = sua['rating'].replace(0.0,  np.nan)
sua['census_code'].dropna(inplace=True)
sua['census_block'].dropna(inplace=True)

In [2]:
lane = sua[sua['county'] == 41039].copy()
lane = lane[lane['test_type'] != 'duplicate']
lane = lane[lane['census_code'].notnull()]
lane = lane[lane['census_block'].notnull()]

# Convert census_codes to integer from float64
lane['census_code'] = lane['census_code'].astype(np.int64)
lane['census_block'] = lane['census_block'].astype(np.int64)

In [3]:
lane.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,638.000000,369.000000,307.000000,290.000000,639.000000,290.000000,410.000000,639.000000,639.000000,639.000000,...,639.000000,639.0,0.0,6.390000e+02,0.0,0.0,639.0,167.000000,0.0,6.390000e+02
mean,97422.982759,77.051491,109.687296,5.817931,51.959828,27.946828,4.034146,0.981221,44.042221,-123.113959,...,20.215102,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1799.538922,NaN,4.103900e+14
std,30.214222,39.361135,171.615926,13.421831,71.280502,125.701847,1.870843,0.135851,0.079901,0.226643,...,67.347328,0.0,NaN,1.592675e+03,NaN,NaN,0.0,10184.125963,NaN,1.592664e+07
min,97401.000000,0.000000,1.000000,0.010000,0.090000,0.040000,1.000000,0.000000,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97402.000000,50.000000,13.000000,0.600000,7.325000,1.170000,3.000000,1.000000,44.028300,-123.147050,...,1.415000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,48.500000,NaN,4.103900e+14
50%,97405.000000,70.000000,60.000000,1.355000,26.580000,3.035000,4.000000,1.000000,44.050000,-123.094100,...,5.720000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,58.000000,NaN,4.103900e+14
75%,97439.000000,90.000000,120.000000,4.500000,68.520000,11.225000,5.000000,1.000000,44.075500,-123.060600,...,11.330000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1062.000000,NaN,4.103900e+14
max,97490.000000,200.000000,1000.000000,120.000000,591.840000,1428.570000,7.000000,1.000000,44.262200,-122.163100,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,118584.000000,NaN,4.103901e+14


In [4]:
lane_tracts = lane['census_code'].unique()
len(lane_tracts)

85

## Census Tract assignments

In [5]:
# Hand built by clicking around on speedupamerica.com's results page
# Next time I will use https://tigerweb.geo.census.gov/tigerweb/
eugene_tracts = [
    41039002201, 41039002202,
    41039002301,
    41039002401, 41039002302, 41039002403, 41039002404,
    41039002501, 41039002503, 41039002504,
    41039002600,
    41039002700,
    41039002800,
    41039002902, 41039002903, 41039002904,
    41039003000,
    41039003101, 41039003102,
    41039003600, # Glenwood and LCC
    41039003700, # UofO
    41039003800,
    41039003900,
    41039004000,
    41039004100,
    41039004200,
    41039004300,
    41039004401, 41039004403, 41039004404, 41039004405,
    41039004501, 41039004502,
    41039004600,
    41039004700,
    41039004800,
    41039004900,
    41039005000,
    41039005100,
    41039005300,
    41039005400
]
springfield_tracts = [
    41039001801, 41039001803, 41039001804, # Thurston
    41039001902, 41039001903, 41039001904, # Centeral Springfield
    41039002001, 41039002002, # North Springfield
    41039002101, 41039002102, # Gateway
    41039003201, 41039003202, # West Springfield (Centenial)
    41039003301, 41039003302, # Springfield
    41039003400, # East of Mowhawl
    41039003500 # Jasper Rd and Dorris Ranch
]

## Eugene

In [6]:
eugene = lane[lane['census_code'].isin(eugene_tracts)]
eugene.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,360.000000,179.000000,158.000000,145.000000,360.000000,145.000000,222.000000,360.000000,360.000000,360.000000,...,360.000000,360.0,0.0,3.600000e+02,0.0,0.0,360.0,106.000000,0.0,3.600000e+02
mean,97402.811111,77.005587,155.525316,3.707103,69.661556,5.871448,4.666667,0.969444,44.056434,-123.106562,...,30.795639,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,2486.311321,NaN,4.103900e+14
std,1.836761,37.046357,211.005651,11.610709,83.393229,14.285555,1.692856,0.172350,0.026602,0.031986,...,87.281750,0.0,NaN,9.054884e+02,NaN,NaN,0.0,12695.264035,NaN,9.055002e+06
min,97401.000000,0.000000,1.000000,0.010000,0.230000,0.040000,1.000000,0.000000,43.993300,-123.200500,...,0.100000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97401.000000,50.000000,50.000000,0.500000,16.077500,0.920000,4.000000,1.000000,44.039750,-123.126675,...,4.492500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,40.500000,NaN,4.103900e+14
50%,97402.000000,70.000000,80.000000,1.000000,43.970000,1.720000,5.000000,1.000000,44.050000,-123.098650,...,5.975000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,65.000000,NaN,4.103900e+14
75%,97405.000000,92.500000,200.000000,2.000000,83.790000,4.110000,6.000000,1.000000,44.074525,-123.086300,...,11.732500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1089.000000,NaN,4.103900e+14
max,97408.000000,200.000000,1000.000000,120.000000,591.840000,134.850000,7.000000,1.000000,44.131400,-123.024100,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,118584.000000,NaN,4.103901e+14


In [7]:
eugene.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating         \
                                                   count      mean median   
provider                                                                    
Comcast Cable Communications, LLC                    228  4.442857    4.0   
XS Media                                              53  5.833333    6.0   
CenturyLink Communications, LLC                       40  3.416667    4.0   
Network for Education and Research in Oregon (N...    10  6.000000    6.0   
Emerald Broadband, LLC                                 5  5.400000    7.0   
AT&T Mobility LLC                                      4  4.000000    4.0   
University of Oregon                                   4  7.000000    7.0   
Verizon Wireless                                       4  3.000000    3.0   
ATT                                                    2       NaN    NaN   
Douglas FastNet                                        2  7.000000    7.0   
Sprint Personal Communications Systems                 2  3.000000    3.0   
Cogent Communications                                  1  7.000000    7.0   
Peak Internet, LLC                                     1       NaN    NaN   
Sprint                                                 1       NaN    NaN   

                                                         actual_down_speed  \
                                                   count              mean   
provider                                                                     
Comcast Cable Communications, LLC                    140         72.337982   
XS Media                                              42        114.635660   
CenturyLink Communications, LLC                       24         14.503000   
Network for Education and Research in Oregon (N...     2         55.042000   
Emerald Broadband, LLC                                 5         29.784000   
AT&T Mobility LLC                                      2          9.022500   
University of Oregon                                   1         44.880000   
Verizon Wireless                                       1         15.920000   
ATT                                                    0         86.675000   
Douglas FastNet                                        2        145.640000   
Sprint Personal Communications Systems                 1         14.455000   
Cogent Communications                                  1        265.150000   
Peak Internet, LLC                                     0          7.630000   
Sprint                                                 0         16.220000   

                                                                   \
                                                     median count   
provider                                                            
Comcast Cable Communications, LLC                    58.760   228   
XS Media                                             63.870    53   
CenturyLink Communications, LLC                       7.700    40   
Network for Education and Research in Oregon (N...   26.120    10   
Emerald Broadband, LLC                               28.820     5   
AT&T Mobility LLC                                     7.205     4   
University of Oregon                                 44.885     4   
Verizon Wireless                                     13.880     4   
ATT                                                  86.675     2   
Douglas FastNet                                     145.640     2   
Sprint Personal Communications Systems               14.455     2   
Cogent Communications                               265.150     1   
Peak Internet, LLC                                    7.630     1   
Sprint                                               16.220     1   

                                                   monthly_price               \
                                                            mean median count   
provider                                                                        


In [8]:
eugene.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
})

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002201     1  5.000000    5.0     1          1.570000    1.570     1   
41039002202    10  3.750000    4.0     4         58.993000   41.810    10   
41039002301     2  3.500000    3.5     2         81.890000   81.890     2   
41039002302     8  5.200000    6.0     5         38.397500   33.625     8   
41039002401     3  4.666667    4.0     3         32.286667   24.330     3   
41039002403     4  4.000000    4.0     2         70.337500   49.365     4   
41039002404     6  3.500000    3.5     4         44.775000   35.000     6   
41039002501     1       NaN    NaN     0        309.340000  309.340     1   
41039002503    13  4.142857    4.0     7        117.247692   45.960    13   
41039002504     1       NaN    NaN     0         57.360000   57.360     1   
41039002600     7  3.333333    3.0     3         17.928571   10.080     7   
41039002700     7  4.800000    5.0     5         46.447143   23.660     7   
41039002800     8  6.000000    7.0     5         44.885000   25.115     8   
41039002902     6  3.666667    4.0     3        100.063333   95.070     6   
41039002903     6  3.500000    3.5     4         44.403333   42.580     6   
41039002904     4  4.000000    4.0     2         39.475000   45.320     4   
41039003000    15  6.000000    6.0     8         58.174667   33.430    15   
41039003101     8  3.833333    4.0     6         42.001250   29.655     8   
41039003102    12  3.333333    3.5     6         42.593333   34.380    12   
41039003600     2  4.500000    4.5     2          4.160000    4.160     2   
41039003700     7  5.000000    4.5     4         71.350000   71.940     7   
41039003800     9  5.142857    5.0     7         41.162222   23.870     9   
41039003900    32  5.315789    6.0    19         75.473125   58.690    32   
41039004000    26  5.545455    6.0    22         56.701538   30.360    26   
41039004100     1  1.000000    1.0     1        160.200000  160.200     1   
41039004200     6  4.250000    4.5     4         66.888333   63.090     6   
41039004300     6  4.000000    4.0     2         87.410000   28.780     6   
41039004401    11  5.000000    5.0    11         53.839091   54.130    11   
41039004403     8  4.666667    4.5     6         65.550000   67.460     8   
41039004404     4  3.333333    4.0     3        103.185000   85.525     4   
41039004405    12  5.200000    6.0     5         55.065833   28.685    12   
41039004501     2  1.500000    1.5     2         12.715000   12.715     2   
41039004502    43  5.227273    6.0    22        114.003256   67.820    43   
41039004600     5       NaN    NaN     0         56.790000   41.680     5   
41039004700    24  4.235294    5.0    17         44.717917   45.185    24   
41039004800     6  3.500000    4.0     6         34.390000   22.090     6   
41039004900     4  5.000000    5.0     3         76.147500   69.280     4   
41039005000     9  3.000000    3.0     6         97.913333   66.340     9   
41039005100     3  6.000000    6.0     2        104.380000   38.370     3   
41039005300     6  5.500000    5.5     2        166.590000  178.065     6   
41039005400    12  5.000000    5.0     6         74.201667   49.260    12   

            monthly_price              actual_price                
                     mean median count         mean  median count  
census_code                                                        
41039002201    120.000000  120.0     1          NaN     NaN     0  
41039002202     89.000000   70.0     5     7.315000   1.345     4  
41039002301     50.000000   50.0     1          NaN     NaN     0  
41039002302     66.250000   70.0     4     1.632500   1.320     4  
41039002401     74.000000   74.0     2          NaN     NaN     0  
41039002403    103.333333  100.0     3     1.215000   1.215     2  
41039002404   

## Springfield

In [9]:
springfield = lane[lane['census_code'].isin(springfield_tracts)]
springfield.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,59.000000,40.000000,37.000000,37.000000,59.000000,37.000000,43.000000,59.000000,59.000000,59.000000,...,59.000000,59.0,0.0,5.900000e+01,0.0,0.0,59.0,20.000000,0.0,5.900000e+01
mean,97477.288136,73.225000,116.945946,3.412162,58.373051,9.137297,3.883721,0.983051,44.056025,-122.989878,...,7.899322,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,498.000000,NaN,4.103900e+14
std,0.456782,31.323365,137.507161,5.597081,58.816810,22.465579,1.802238,0.130189,0.014284,0.045462,...,8.094987,0.0,NaN,6.653510e+02,NaN,NaN,0.0,1464.151775,NaN,6.653677e+06
min,97477.000000,13.000000,3.000000,0.030000,0.440000,0.520000,1.000000,0.000000,44.011700,-123.043000,...,0.260000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,11.000000,NaN,4.103900e+14
25%,97477.000000,45.750000,24.000000,0.570000,16.665000,0.830000,2.500000,1.000000,44.046450,-123.022000,...,4.390000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,29.000000,NaN,4.103900e+14
50%,97477.000000,70.000000,60.000000,0.800000,33.970000,2.110000,4.000000,1.000000,44.052700,-123.000000,...,5.750000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
75%,97478.000000,89.250000,150.000000,2.000000,73.940000,5.820000,5.000000,1.000000,44.068100,-122.967400,...,11.305000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,65.000000,NaN,4.103900e+14
max,97478.000000,161.000000,500.000000,26.670000,260.390000,125.000000,7.000000,1.000000,44.078400,-122.874300,...,40.510000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,5985.000000,NaN,4.103900e+14


In [10]:
springfield.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating               \
                                  count      mean median count   
provider                                                         
Comcast Cable Communications, LLC    44  4.375000    4.5    32   
CenturyLink Communications, LLC      10  2.714286    2.0     7   
AVAST Software s.r.o.                 1  2.000000    2.0     1   
SoftLayer Technologies Inc.           1  1.000000    1.0     1   
Sprint                                1  3.000000    3.0     1   
Verizon Wireless                      1       NaN    NaN     0   
ViaSat,Inc.                           1  2.000000    2.0     1   

                                  actual_down_speed               \
                                               mean median count   
provider                                                           
Comcast Cable Communications, LLC         73.038409  61.26    44   
CenturyLink Communications, LLC            9.147000   6.66    10   
AVAST Software s.r.o.                     31.720000  31.72     1   
SoftLayer Technologies Inc.               23.900000  23.90     1   
Sprint                                    76.380000  76.38     1   
Verizon Wireless                           3.460000   3.46     1   
ViaSat,Inc.                                3.390000   3.39     1   

                                  monthly_price              actual_price  \
                                           mean median count         mean   
provider                                                                    
Comcast Cable Communications, LLC     72.892857   70.0    28     2.281111   
CenturyLink Communications, LLC       77.222222   75.0     9    30.068750   
AVAST Software s.r.o.                 60.000000   60.0     1          NaN   
SoftLayer Technologies Inc.           13.000000   13.0     1     0.540000   
Sprint                                      NaN    NaN     0          NaN   
Verizon Wireless                            NaN    NaN     0          NaN   
ViaSat,Inc.                          120.000000  120.0     1    35.400000   

                                                
                                  median count  
provider                                        
Comcast Cable Communications, LLC   1.33    27  
CenturyLink Communications, LLC    10.94     8  
AVAST Software s.r.o.                NaN     0  
SoftLayer Technologies Inc.         0.54     1  
Sprint                               NaN     0  
Verizon Wireless                     NaN     0  
ViaSat,Inc.                        35.40     1

In [11]:
springfield.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002002     9  4.000000    3.0     9         51.726667   56.030     9   
41039003302     7  3.428571    3.0     7         37.878571   26.580     7   
41039001801     5  4.250000    5.0     4         40.952000    8.380     5   
41039002102     5  4.500000    4.5     2         56.196000   33.970     5   
41039002001     4  3.250000    3.0     4         55.325000   38.185     4   
41039003201     4  3.000000    3.0     1         28.467500   28.270     4   
41039003202     4  2.750000    2.0     4        112.510000   82.875     4   
41039001803     3  5.000000    5.0     2         83.553333   78.810     3   
41039001804     3  2.500000    2.5     2         81.046667   10.780     3   
41039002101     3  7.000000    7.0     1        115.613333  151.460     3   
41039003301     3  4.000000    4.0     1         38.986667   33.340     3   
41039003500     3  4.000000    4.0     1         56.016667   67.320     3   
41039001902     2  6.500000    6.5     2         80.055000   80.055     2   
41039001903     2  3.500000    3.5     2         40.860000   40.860     2   
41039001904     1       NaN    NaN     0          3.460000    3.460     1   

            monthly_price              actual_price                
                     mean median count         mean  median count  
census_code                                                        
41039002002     53.875000   45.0     8     1.570000   0.730     8  
41039003302     85.666667   87.0     6    13.014000   3.200     5  
41039001801     97.000000   75.0     3    47.733333   9.250     3  
41039002102     68.250000   74.0     4     4.582500   2.585     4  
41039002001     76.333333   69.0     3     2.150000   2.150     1  
41039003201     41.666667   50.0     3     1.476667   1.490     3  
41039003202     94.333333  130.0     3     0.843333   0.540     3  
41039001803     90.000000   90.0     1     7.820000   7.820     1  
41039001804     82.500000   82.5     2    28.385000  28.385     2  
41039002101     70.000000   70.0     1     4.340000   4.340     1  
41039003301           NaN    NaN     0          NaN     NaN     0  
41039003500     82.500000   82.5     2     6.690000   6.690     2  
41039001902     80.000000   80.0     1     2.550000   2.550     1  
41039001903     72.500000   72.5     2     2.000000   2.000     2  
41039001904           NaN    NaN     0          NaN     NaN     0

## Lane County

In [12]:
county_tracts = np.setdiff1d(lane_tracts, eugene_tracts)
county_tracts = np.setdiff1d(county_tracts, springfield_tracts)

county = lane[lane['census_code'].isin(county_tracts)]
county.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,219.000000,150.000000,112.00000,108.000000,220.000000,108.000000,145.000000,220.0,220.000000,220.000000,...,220.000000,220.0,0.0,2.200000e+02,0.0,0.0,220.0,41.000000,0.0,2.200000e+02
mean,97441.511416,78.126667,42.62500,9.476111,21.273455,64.029074,3.110345,1.0,44.015263,-123.159341,...,6.204364,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,658.878049,NaN,4.103900e+14
std,29.102060,43.874791,67.37165,16.524226,31.834111,200.339871,1.764296,0.0,0.127558,0.375651,...,16.909151,0.0,NaN,7.269570e+02,NaN,NaN,0.0,1704.056545,NaN,7.269375e+06
min,97402.000000,20.000000,1.00000,0.200000,0.090000,0.350000,1.000000,1.0,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,10.000000,NaN,4.103900e+14
25%,97419.000000,49.250000,10.00000,1.397500,2.645000,3.945000,1.000000,1.0,43.942725,-123.339150,...,0.600000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
50%,97437.000000,65.000000,17.50000,3.775000,8.110000,10.700000,3.000000,1.0,44.031350,-123.161800,...,1.660000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
75%,97463.000000,90.000000,60.00000,8.372500,22.102500,26.125000,4.000000,1.0,44.093300,-123.006500,...,5.962500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,144.000000,NaN,4.103900e+14
max,97490.000000,200.000000,400.00000,90.000000,176.520000,1428.570000,7.000000,1.0,44.262200,-122.163100,...,203.650000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,9752.000000,NaN,4.103901e+14


In [13]:
county.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating               \
                                       count      mean median count   
provider                                                              
CenturyLink Communications, LLC           89  2.531250    2.0    64   
Charter Communications                    45  4.444444    5.0    27   
Verizon Wireless                          21  2.272727    2.0    11   
Comcast Cable Communications, LLC         13  4.500000    4.5     8   
ViaSat,Inc.                               13  2.111111    2.0     9   
Hughes Network Systems                     9  2.400000    3.0     5   
Emerald Broadband, LLC                     8  3.142857    3.0     7   
XS Media                                   8  4.125000    4.0     8   
Hunter Communications                      6  2.750000    3.0     4   
AT&T Mobility LLC                          3       NaN    NaN     0   
LS Networks                                2  5.500000    5.5     2   
Sprint                                     1       NaN    NaN     0   
Sprint Personal Communications Systems     1       NaN    NaN     0   
tfm associates                             1       NaN    NaN     0   

                                       actual_down_speed                 \
                                                    mean   median count   
provider                                                                  
CenturyLink Communications, LLC                 6.569213    3.990    89   
Charter Communications                         43.865556   33.020    45   
Verizon Wireless                               18.085238    7.730    21   
Comcast Cable Communications, LLC              62.991538   53.910    13   
ViaSat,Inc.                                     7.788462    5.860    13   
Hughes Network Systems                          5.138889    2.100     9   
Emerald Broadband, LLC                         18.511250    6.115     8   
XS Media                                       19.925000   19.790     8   
Hunter Communications                          36.315000    5.710     6   
AT&T Mobility LLC                              12.403333    6.710     3   
LS Networks                                    11.450000   11.450     2   
Sprint                                        115.560000  115.560     1   
Sprint Personal Communications Systems          2.920000    2.920     1   
tfm associates                                 71.400000   71.400     1   

                                       monthly_price               \
                                                mean median count   
provider                                                            
CenturyLink Communications, LLC            62.603175   55.0    63   
Charter Communications                     61.111111   64.0    27   
Verizon Wireless                          117.333333  107.5    12   
Comcast Cable Communications, LLC          73.000000   80.0     9   
ViaSat,Inc.                               121.384615  105.0    13   
Hughes Network Systems                     90.000000   92.5     8   
Emerald Broadband, LLC                    114.285714   50.0     7   
XS Media                                  118.800000  119.0     5   
Hunter Communications                      71.500000   62.5     4   
AT&T Mobility LLC                                NaN    NaN     0   
LS Networks                                41.000000   41.0     2   
Sprint                                           NaN    NaN     0   
Sprint Personal Communications Systems           NaN    NaN     0   
tfm associates                                   NaN    NaN     0   

                                       actual_price                 
                                               mean   median count  
provider                                                            
CenturyLink Communications, LLC           63.801458   16.495    48  
Charter Communications                     3.844737    1.380    19  
Verizon Wireless                       

In [14]:
county.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                \
            count      mean median count              mean  median count   
census_code                                                                
41039000100    21  2.857143    2.5    14          9.718571   6.650    21   
41039001002    20  2.000000    1.0    15          9.387500   5.395    20   
41039000902    17  4.250000    4.5    12         17.071765   4.420    17   
41039000903    16  3.727273    3.0    11         11.718750   2.715    16   
41039001101    13  2.800000    3.0     5         23.373846  10.130    13   
41039000200    13  2.100000    2.0    10         10.287692   5.820    13   
41039000904    12  2.750000    2.0     8         22.580833  10.070    12   
41039000402    10  2.666667    3.0     9          6.893000   3.490    10   
41039000403     9  4.000000    4.0     4         36.667778   7.510     9   
41039000800     8  3.000000    3.0     7          6.183750   2.185     8   
41039001201     8  3.000000    3.0     5         25.893750   5.045     8   
41039001700     7  2.500000    2.5     2         38.901429   8.700     7   
41039001500     7  3.600000    2.0     5         23.137143   7.140     7   
41039001102     7  3.000000    4.0     5         44.821429  24.790     7   
41039000707     6  4.000000    3.0     3         40.711667  38.070     6   

            monthly_price              actual_price                 
                     mean median count         mean   median count  
census_code                                                         
41039000100     53.000000   48.5    16    12.733077   15.380    13  
41039001002    124.411765  119.0    17   198.441250   31.335     8  
41039000902     87.142857   85.0    14   106.293000   19.610    10  
41039000903     60.181818   58.0    11    79.232857   10.630     7  
41039001101     78.500000   60.0     4    19.386667   16.670     3  
41039000200     87.222222   70.0     9    13.660000    5.450     7  
41039000904     82.100000   72.5    10     7.673750    5.960     8  
41039000402     91.600000   77.5    10   198.492500   18.415     8  
41039000403     66.666667   50.0     3    19.875000   19.875     2  
41039000800     80.714286   80.0     7   224.890000  145.255     4  
41039001201     71.200000   66.0     5     6.506667    7.940     3  
41039001700     58.000000   55.5     4     6.400000    6.060     3  
41039001500     64.750000   59.5     4    13.912500    8.945     4  
41039001102     73.500000   63.5     4    32.050000   27.140     4  
41039000707     82.500000   82.5     2     1.540000    1.540     2